In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import os, glob

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten

In [10]:
train_img_list = glob.glob("fingers/train/*.png")
test_img_list = glob.glob("fingers/test/*.png")
print(len(train_img_list),
     len(test_img_list), sep = '\n')

18000
3600


In [34]:
def import_data():
    train_img_data = []
    test_img_data = []
    train_label_data = []
    test_label_data = []
    
    for img in train_img_list:
        img_read = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        img_read = cv2.resize(img_read, (128,128))
        train_img_data.append(img_read)
        train_label_data.append(int(img[-6]))
    
    for img in test_img_list:
        img_read = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        img_read = cv2.resize(img_read, (128,128))
        test_img_data.append(img_read)
        test_label_data.append(int(img[-6]))
        
    return np.array(train_img_data), np.array(test_img_data), np.array(train_label_data), np.array(test_label_data)

In [35]:
xtrain, xtest, ytrain, ytest = import_data()

In [36]:
ytrain.shape

(18000,)

In [37]:
from keras.utils import to_categorical 
xtrain = xtrain.reshape(xtrain.shape[0], 128, 128, 1)
xtest = xtest.reshape(xtest.shape[0], 128, 128, 1)

ytrain = to_categorical(ytrain, 6)
ytest = to_categorical(ytest, 6)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(18000, 128, 128, 1) (3600, 128, 128, 1) (18000, 6) (3600, 6)


In [38]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (128, 128, 1), activation = 'relu'))
model.add(Conv2D(32, (3,3), activation = 'relu'))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Conv2D(64, (3,3), activation = 'relu'))

model.add(MaxPool2D((2,2)))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(Conv2D(128, (3,3), activation = 'relu'))

model.add(Flatten())

model.add(Dropout(0.40))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.40))
model.add(Dense(6, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 122, 122, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 120, 120, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 58, 58, 128)       73856     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 56, 56, 128)       147584    
__________

In [ ]:
model.compile('adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x = xtrain, y = ytrain, batch_size = 128, epochs = 10, validation_data = (xtest, ytest))

Train on 18000 samples, validate on 3600 samples
Epoch 1/10


In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(2,1,figsize=(10,10))
ax[0].plot(history.history['loss'], color='r', label='Training Loss')
ax[0].plot(history.history['val_loss'], color='g', label='Validation Loss')
ax[0].legend(loc='best',shadow=True)
ax[0].grid(True)

ax[1].plot(history.history['acc'], color='r', label='Training Accuracy')
ax[1].plot(history.history['val_acc'], color='g', label='Validation Accuracy')
ax[1].legend(loc='best',shadow=True)
ax[1].grid(True)
plt.savefig("result+"+str(nb_epoches)+".png")
plt.show()